In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [2]:
# X=np.load("X (1).npy")
# Y=np.load("Y (1).npy")

# Load the data
X = np.load("X_reg.npy")
Y = np.load("Y_reg.npy")

Y = np.log10(Y)


from sklearn.preprocessing import RobustScaler

sc_y = RobustScaler().fit(Y.reshape(-1, 1))
Y = sc_y.transform(Y.reshape(-1, 1))
Y = Y.reshape(-1,)

In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [4]:
input_size = 101
hidden_size = 200
num_classes = 1
num_epochs = 5
batch_size = 1000
learning_rate = 0.001

In [8]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.tanh = nn.Tanh()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        return out

In [9]:
model = Net(input_size,hidden_size,num_classes)
model.cuda()
criterion = nn.L1Loss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
model = torch.load("DNN_New.pyt")

AttributeError: Can't get attribute '_rebuild_tensor_v2' on <module 'torch._utils' from '/anaconda3/lib/python3.6/site-packages/torch/_utils.py'>

### To Load, uncomment the below line and do not train !
model = torch.load("DNN_New.pyt")

In [6]:
epochs = 4000

In [7]:
# test
losses=[]
for epoch in range(epochs):
    
    epoch +=1
    
    for x,y in zip(X_train,Y_train):
        y=np.array([y])
        x=torch.from_numpy(np.expand_dims(x, axis=0))
        y=torch.from_numpy(y)
        
        inputs = Variable(x.type(torch.cuda.FloatTensor))
        labels = Variable(y.type(torch.cuda.FloatTensor))
        
        #clear grads
        optimizer.zero_grad()
        #forward to get predicted values
        outputs = torch.squeeze(model.forward(inputs))
        
            
        loss = criterion(outputs, labels)
        

        losses.append(float(loss))
        loss.backward()# back props
        optimizer.step()# update the parameters
    print('epoch {}, loss {}'.format(epoch,loss.data[0]))

AssertionError: Torch not compiled with CUDA enabled

## Evaluate the performance on the training data

In [29]:
model.eval()

Net(
  (fc1): Linear(in_features=101, out_features=200, bias=True)
  (tanh): Tanh()
  (fc2): Linear(in_features=200, out_features=1, bias=True)
)

In [30]:
from sklearn.metrics import r2_score

In [31]:
# X_train and Y_train
model_pred=model(Variable(torch.from_numpy(X_train).type(torch.cuda.FloatTensor)))

r2_score(Y_train, model_pred.cpu().detach().numpy())

0.9882534807409923

## Evaluate the model on the testing data

In [42]:
# X_test and Y_test
model_pred_test=model(Variable(torch.from_numpy(X_test).type(torch.cuda.FloatTensor)))

In [33]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [34]:
# covert to numpy
model_pred_test = model_pred_test.cpu().detach().numpy()

In [35]:
r2_score(Y_test, model_pred_test)

0.5204406029761776

In [36]:
mean_absolute_error(Y_test, model_pred_test)

0.4800216345638305

In [38]:
from math import sqrt

In [39]:
sqrt(mean_squared_error(Y_test,model_pred_test))

0.6029675993571644

In [41]:
torch.save(model.state_dict(),"DNN_New.pyt")